<a href="https://colab.research.google.com/github/Pooya448/Tumor_Segmentation/blob/main/Patch_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf ~/.cache/
!rm -rf /content/sample_data

In [ ]:
!pip install -U --no-cache-dir gdown --pre

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.12.2-cp36-none-any.whl size=9694 sha256=cc72192a232a6397ec574c235cea585d2bd6dd5207f863fa5dd1fa071ea91d04
  Stored in directory: /tmp/pip-ephem-wheel-cache-iegvg0_n/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


In [ ]:
### Output
!gdown --id 1-NxnJmIB8q5ov5bseg5mBqbUPsH_WUlM
!gdown --id 1-MMz-2C2uSBSiYaZ9ZLCrdIzDjlTS37o

Downloading...
From: https://drive.google.com/uc?id=1-NxnJmIB8q5ov5bseg5mBqbUPsH_WUlM
To: /content/train_labels.json
3.85MB [00:00, 286MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-MMz-2C2uSBSiYaZ9ZLCrdIzDjlTS37o
To: /content/dev_labels.json
100% 1.18M/1.18M [00:00<00:00, 172MB/s]


In [ ]:
### Dev
!gdown --id 1-4uxCjaWTqzbf0tM8vspEIKUny1AKK6T

Downloading...
From: https://drive.google.com/uc?id=1-4uxCjaWTqzbf0tM8vspEIKUny1AKK6T
To: /content/dev_patch_data.zip
2.81GB [00:16, 175MB/s]


In [ ]:
### Train
!gdown --id 1-8ZPLkZpjRHBwY_k0k0Ck9H_8wb9Z3a4

Downloading...
From: https://drive.google.com/uc?id=1-8ZPLkZpjRHBwY_k0k0Ck9H_8wb9Z3a4
To: /content/train_patch_data.zip
8.83GB [01:08, 129MB/s]


In [ ]:
!unzip /content/dev_patch_data.zip
!unzip /content/train_patch_data.zip

In [1]:
import numpy as np
import tensorflow as tf
import os
import cv2
from tensorflow import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import json

In [2]:
with open('train_labels.json') as json_file: 
    train_label = json.load(json_file)
    train_count = len(train_label)
    print(train_count)
with open('dev_labels.json') as json_file: 
    dev_label = json.load(json_file)
    dev_count = len(dev_label)
    print(dev_count)

304810
99076


In [3]:
BATCH_SIZE = 1024
INPUT_SHAPE = (256, 256, 3)

STEPS_PER_EPOCH = train_count // BATCH_SIZE
VALIDATION_STEPS = dev_count // BATCH_SIZE

EPOCHS = 10

In [4]:
def load_images(filename, label):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.reshape(image, [256, 256, 3])

    label = tf.reshape(label, [1])
    return image, label

In [5]:
train_dir = '/content/content/Data/train'
dev_dir = '/content/content/Data/dev'

def create_dataset(dir, label_dic, is_training=True):
    filepaths = list(tf.data.Dataset.list_files(dir + '/*.jpg'))

    labels = []

    for f in filepaths:
        ind = f.numpy().decode().split('/')[-1].split('.')[0]
        labels.append(label_dic[ind])

    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    ds = ds.map(load_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.cache() 

    if is_training:
        ds = ds.shuffle(len(filepaths), reshuffle_each_iteration=True)
        ds = ds.repeat(EPOCHS) 
    ds = ds.batch(BATCH_SIZE) 
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

    return ds


train_ds = create_dataset(train_dir, train_label)
dev_ds = create_dataset(dev_dir, dev_label, False)


In [15]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.69.71.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.71.114:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [16]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [17]:
def create_model(input_shape, batch_size):
    VGG16 = keras.applications.VGG16(include_top=False,input_shape=input_shape, weights='imagenet')

    for layer in VGG16.layers:
        layer.trainable = False

    input_layer = keras.Input(shape=input_shape, batch_size=batch_size)

    VGG_out = VGG16(input_layer)

    x = Flatten(name='flatten', input_shape=(512,8,8))(VGG_out)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid', name='fc2')(x)

    model = Model(input_layer, x)
    model.summary()
    return model

In [19]:
adam = tf.keras.optimizers.Adagrad(learning_rate=0.1)
binary_loss = tf.keras.losses.BinaryCrossentropy



with strategy.scope():

    model = create_model(INPUT_SHAPE, BATCH_SIZE)
    model.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])
    
    model.fit(train_ds,
          epochs=5,
          steps_per_epoch=STEPS_PER_EPOCH,
          validation_data=dev_ds,
          validation_steps=VALIDATION_STEPS
        )


Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(128, 256, 256, 3)]      0         
_________________________________________________________________
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (128, 32768)              0         
_________________________________________________________________
fc1 (Dense)                  (128, 256)                8388864   
_________________________________________________________________
dropout_5 (Dropout)          (128, 256)                0         
_________________________________________________________________
fc2 (Dense)                  (128, 1)                  257       
Total params: 23,103,809
Trainable params: 8,389,121
Non-trainable params: 14,714,688
_________________________________

InvalidArgumentError: ignored

In [13]:
# checkpoint_filepath = '/checkpoint'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=False,
#     monitor='val_acc',
#     mode='auto',
#     save_best_only=True)

In [14]:
#   callbacks=[model_checkpoint_callback]
#   validation_data=dev_ds, 
#   validation_steps=VALIDATION_STEPS
#   steps_per_epoch=STEPS_PER_EPOCH
# model.fit(train_ds,
#           epochs=5,
#           steps_per_epoch=STEPS_PER_EPOCH,
#           validation_data=dev_ds, 
#           validation_steps=VALIDATION_STEPS)